# Example notebook 

This notebook is a copy from `examples/getting_started/0_Getting_started/0_Getting_started.ipynb` with a small modifications for use with [Papermill](https://github.com/nteract/papermill).


## Papermill parameters

Papermill is a tool for parameterizing, executing, and analyzing Jupyter Notebooks. 
To parameterize a notebook, we need tag a cell that contains all the parameters. 
We can only tag one cell, and for simplicity we place it at the top of the notebook. 
Full instructions are available here https://papermill.readthedocs.io/en/latest/usage-parameterize.html#designate-parameters-for-a-cell. 

Below, we tag the parameter `shots=1000`. 
We can then run the notebook as usual cell-by-cell to see the results. 

We additionally add two Braket specific parameters called device_arn and results_dir. 
These will be overriden when we run a Braket Job. 

**In the following cell, we added a tag called `parameters` for Papermill to recognize. If you dont see the tag, follow the instructions above.**

In [ ]:
# Papermill parameters
shots = 1000

# Braket specific parameters
device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
results_dir = "."

# Getting started with Amazon Braket

In this hello-world tutorial we prepare a maximally entangled Bell state between two qubits. We then run our circuit on a local simulator and obtain the results.

In [ ]:
# general imports
import matplotlib.pyplot as plt

%matplotlib inline

from braket.aws import AwsDevice

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit
from braket.devices import LocalSimulator

## Build a circuit

Let's build a Bell state with two qubits. By calling `Circuit()` we create an empty circuit, and we can just add gates to the circuit. 

In [ ]:
# build a Bell state with two qubits. Here 'cnot(control=0, target=1)' can be simplified as 'cnot(0,1)'
bell = Circuit().h(0).cnot(control=0, target=1)

## Submit the circuit to the local simulator and obtain the results

Here we submit our circuit to the local simulator and obtain the results.

In [ ]:
# set up device
if "local" in device_arn:
    device = LocalSimulator()
else:
    device = AwsDevice(device_arn)

# run circuit
result = device.run(bell, shots=shots).result()
# get measurement shots
counts = result.measurement_counts
counts = {k: counts[k] for k in sorted(counts)}  # sort results
print(counts)

**The following cell saves the results to a file**

The use of `results_dir` is that any results are saved to the correct directory when called from a Braket Job. Below, we save the measurement counts to a .pkl file. 

In [ ]:
import pickle

# save meaurement counts to a file
with open(f"{results_dir}/results.pkl", "wb") as f:
    pickle.dump(counts, f)


# re-load the results with
# with open(f"{results_dir}/results.pkl", 'rb') as f:
# counts = pickle.load(f)

**The following cell saves the figure to a file**

In [ ]:
# plot using Counter
plt.bar(counts.keys(), counts.values())
plt.xlabel("bitstrings")
plt.ylabel("counts")

plt.savefig(f"{results_dir}/histogram.png")